In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import cv2
import numpy
import imageio
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from keras import backend as K
import sys
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [0]:
image_rows, image_columns, image_depth = 64, 64, 18

In [0]:
training_list = []
painpath = '/content/drive/My Drive/Training Data/pain'
Nopainpath = '/content/drive/My Drive/Training Data/Nopain'

In [0]:
directorylisting = os.listdir(painpath)

In [0]:
for video in os.listdir(painpath):
  videopath = os.path.join(painpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  training_list.append(videoarray)

In [7]:
len(training_list)

6

In [0]:
directorylisting = os.listdir(Nopainpath)

In [0]:
for video in os.listdir(Nopainpath):
  videopath = os.path.join(Nopainpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  training_list.append(videoarray)

In [10]:
len(training_list)

12

In [0]:
training_list = numpy.asarray(training_list)
trainingsamples = len(training_list)
traininglabels = numpy.zeros((trainingsamples, ), dtype = int)

In [0]:
traininglabels[0:6] = 0
traininglabels[6:12] = 1

In [0]:
traininglabels = np_utils.to_categorical(traininglabels, 2)

In [0]:
training_data = [training_list, traininglabels]
(trainingframes, traininglabels) = (training_data[0], training_data[1])
training_set = numpy.zeros((trainingsamples, 1, image_rows, image_columns, image_depth))

In [15]:
training_set.shape

(12, 1, 64, 64, 18)

In [0]:
for h in range(trainingsamples):
    training_set[h][0][:][:][:] = trainingframes[h, :, :, :]

In [0]:
training_set = training_set.astype('float32')
training_set -= numpy.mean(training_set)
training_set /= numpy.max(training_set)

In [18]:
model = Sequential()
model.add(Convolution3D(4, (3, 3, 15), input_shape=(1, image_rows, image_columns, image_depth), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, init='normal'))
model.add(Activation('softmax'))

W0315 05:20:10.197818 140366304855936 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0315 05:20:10.209686 140366304855936 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0315 05:20:10.215091 140366304855936 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0315 05:20:10.226079 140366304855936 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0315 05:20:10.226974 140366304855936 module_wrapper

In [20]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 4, 62, 62, 4)      544       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 20, 20, 1)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 20, 20, 1)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 3202      
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 3,746
Trainable params: 3,746
Non-trainable params: 0
_________________________________________________________________


In [0]:
testing_list = []
painpath = '/content/drive/My Drive/TestingData/pain'
Nopainpath = '/content/drive/My Drive/TestingData/Nopain'

In [0]:
for video in os.listdir(painpath):
  videopath = os.path.join(painpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  testing_list.append(videoarray)

In [0]:
for video in os.listdir(Nopainpath):
  videopath = os.path.join(Nopainpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  testing_list.append(videoarray)

In [24]:
len(testing_list)

10

In [0]:
testing_list = numpy.asarray(testing_list)
testingsamples = len(testing_list)
testinglabels = numpy.zeros((testingsamples, ), dtype = int)

In [0]:
testinglabels[0:5] = 0
testinglabels[5:10] = 1

In [0]:
testinglabels = np_utils.to_categorical(testinglabels, 2)

In [0]:
testing_data = [testing_list, testinglabels]
(testingframes, testinglabels) = (testing_data[0], testing_data[1])
testing_set = numpy.zeros((testingsamples, 1, image_rows, image_columns, image_depth))

In [29]:
testing_set.shape

(10, 1, 64, 64, 18)

In [0]:
for h in range(testingsamples):
    testing_set[h][0][:][:][:] = testingframes[h, :, :, :]

In [0]:
testing_set = testing_set.astype('float32')
testing_set -= numpy.mean(testing_set)
testing_set /= numpy.max(testing_set)

In [32]:
hist = model.fit(training_set, traininglabels, epochs = 15, validation_data = (testing_set, testinglabels), shuffle=True)

W0315 05:23:21.100992 140366304855936 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0315 05:23:21.191500 140366304855936 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0315 05:23:21.208056 140366304855936 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 12 samples, validate on 10 samples
Epoch 1/15
12/12 [==============================] - 1s 89ms/step - loss: 0.7905 - acc: 0.4167 - val_loss: 0.6543 - val_acc: 0.6000
Epoch 2/15
12/12 [==============================] - 1s 57ms/step - loss: 0.5915 - acc: 0.7500 - val_loss: 0.6217 - val_acc: 0.6000
Epoch 3/15
12/12 [==============================] - 1s 57ms/step - loss: 0.6897 - acc: 0.4167 - val_loss: 0.5864 - val_acc: 0.9000
Epoch 4/15
12/12 [==============================] - 1s 57ms/step - loss: 0.5294 - acc: 0.6667 - val_loss: 0.5630 - val_acc: 0.9000
Epoch 5/15
12/12 [==============================] - 1s 57ms/step - loss: 0.5167 - acc: 0.7500 - val_loss: 0.5417 - val_acc: 0.9000
Epoch 6/15
12/12 [==============================] - 1s 56ms/step - loss: 0.5179 - acc: 0.7500 - val_loss: 0.5201 - val_acc: 0.9000
Epoch 7/15
12/12 [==============================] - 1s 58ms/step - loss: 0.4296 - acc: 0.8333 - val_loss: 0.4900 - val_acc: 0.9000
Epoch 8/15
12/12 [=====================

In [0]:
predictions = model.predict(testing_set)
predictions_labels = numpy.argmax(predictions, axis=1)

In [34]:
predictions_labels

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [0]:
testinglabels = numpy.argmax(testinglabels, axis=1)

In [36]:
testinglabels

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [38]:
cfm = confusion_matrix(testinglabels, predictions_labels)
print (cfm)

[[5 0]
 [0 5]]
